In [ ]:
import os
import re
import sqlite3
import datetime
import pandas as pd
from IPython.display import display
from Utils import parse_date, FMID, simple_date_pattern, year_pattern, parse_complex_date, ZIP, zip_code_db, parse_time,\
    validate_zip_code

# we should convert this notebook to open refine and then convert the steps to workflow using yes workflow

S_TIME = "Season4Time"
S_TIME_DICT = "Season4TimeDict" # remember to use pickl
S_TIME_LIST = "Season4TimeList" # remember to use pickl
FILE_NAME = "cl_farmers_s4time.csv"
WEEKDAYS = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]

df = pd.read_csv(os.path.join("..", "dataset", "input", "farmers_market.csv"))
print("df.shape[0]", df.shape[0])
assert df[FMID].nunique() == df.shape[0]

Setting up basic util data frames

In [ ]:
time = df[[FMID, S_TIME, ZIP]].dropna()
time[S_TIME] = time[S_TIME].astype(str)
time[ZIP] = time[ZIP].astype(str)
# should have valid zip code
print("invalid zip code row count", len(time.loc[time[ZIP].apply(validate_zip_code) == False]))
time = time.loc[time[ZIP].apply(validate_zip_code) == True]

print("Number of FMIDs with improper timings(without ;)", len(time.loc[time[S_TIME].str.contains(";") == False]))
time = time.loc[time[S_TIME].str.contains(";") == True] # should contain ";"
time = time.loc[time[S_TIME].str.replace(" ", "").str.isalpha() == False] # should contain some numeric data and should not be completely alphabetical
# print(time.shape[0])

In [ ]:
time[S_TIME_LIST] = time.apply(lambda r: r[S_TIME].split(";"), axis=1)
display(time)

In [ ]:
time[S_TIME_DICT] = time.apply(parse_time, axis=1, args=(S_TIME_LIST,))

Defining columns for weekdays

In [ ]:
for weekday in WEEKDAYS:
    time[weekday+"_start"] = [""]*time.shape[0]
    time[weekday+"_end"] = [""]*time.shape[0]

In [ ]:
def flatten_data(r: pd.Series):
    data = r[S_TIME_DICT]
    for key, value in data.items():
        r[key+"_start"] = value[0]
        r[key+"_end"] = value[1]
    return r

time = time.apply(flatten_data, axis=1)    

In [ ]:
display(time)
df_to_save = time.drop([S_TIME, S_TIME_LIST, S_TIME_DICT], axis=1)

In [ ]:
# df_to_save.to_csv(os.path.join("..", "dataset", "output", FILE_NAME))
conn = sqlite3.connect(f"{DATASET_LOC}/../database/farmers_market.db")
df_to_save.to_sql("fm_seasons_times", conn, if_exists="replace")
conn.cursor().execute(
    """
    select count(*) from farmers_market
    """).fetchall()